In [1]:
import json

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [19]:
import pandas as pd
# Read the CSV file
csv_file = "green_tripdata_2019-10.csv"
df = pd.read_csv(csv_file)

df.describe()

/var/folders/vd/dl48s8js5_z0k6_rb5jv19xw0000gn/T/ipykernel_25262/3480047254.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


,VendorID,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
count,387007.000000,387007.000000,476386.000000,476386.000000,387007.000000,476386.000000,476386.000000,476386.000000,476386.000000,476386.000000,476386.000000,0.0,476386.000000,476386.000000,387007.000000,387005.000000,387007.000000
mean,1.833509,1.102104,108.789746,129.697575,1.308302,3.543614,15.782716,0.844550,0.486164,0.983024,0.317710,NaN,0.240391,18.986092,1.454666,1.023294,0.451301
std,0.372521,0.727587,71.420448,76.351002,0.967869,4.168409,14.100716,1.135181,0.089725,1.981928,2.177628,NaN,0.121327,15.621042,0.522706,0.150837,1.018531
min,1.000000,1.000000,1.000000,1.000000,0.000000,-6.930000,-200.000000,-4.500000,-0.500000,-80.800000,-12.500000,NaN,-0.300000,-200.000000,1.000000,1.000000,-2.750000
25%,2.000000,1.000000,55.000000,65.000000,1.000000,1.100000,7.000000,0.000000,0.500000,0.000000,0.000000,NaN,0.300000,9.000000,1.000000,1.000000,0.000000
50%,2.000000,1.000000,82.000000,129.000000,1.000000,2.070000,11.500000,0.500000,0.500000,0.000000,0.000000,NaN,0.300000,14.160000,1.000000,1.000000,0.000000
75%,2.000000,1.000000,166.000000,193.000000,1.000000,4.270000,20.730000,1.000000,0.500000,1.650000,0.000000,NaN,0.300000,24.620000,2.000000,1.000000,0.000000
max,2.000000,99.000000,265.000000,265.000000,9.000000,515.890000,2877.500000,8.250000,0.500000,158.520000,935.500000,NaN,0.300000,2878.300000,5.000000,2.000000,2.750000


In [16]:
from kafka.admin import KafkaAdminClient, NewTopic

admin_client = KafkaAdminClient(
    bootstrap_servers="localhost:9092",  # Change to your Kafka broker
    client_id='admin-client'
)

# Define the topic name
topic_name = "green-trips"

# Delete the topic
admin_client.delete_topics(topics=[topic_name])

new_topic = NewTopic(
    name=topic_name,
    num_partitions=1,  # Change as needed
    replication_factor=1  # Adjust based on your Kafka setup
)

# Create the topic
admin_client.create_topics(new_topics=[new_topic])


print(f"Topic '{topic_name}' created successfully.")

admin_client.close()


Topic 'green-trips' created successfully.


In [17]:
import pandas as pd
from kafka import KafkaProducer
import json
from time import time

# Define the topic name
topic_name = "green-trips"

# Initialize Kafka producer
producer = KafkaProducer(
    bootstrap_servers='localhost:9092',  # Change this if your Kafka is running elsewhere
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Read the CSV file
csv_file = "green_tripdata_2019-10.csv"
df = pd.read_csv(csv_file)

# Select only the required columns
df = df[[
    'lpep_pickup_datetime',
    'lpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'tip_amount'
]]

# Handle missing values while preserving data types
df['lpep_pickup_datetime'] = df['lpep_pickup_datetime'].fillna('1970-01-01 00:00:00').astype(str)
df['lpep_dropoff_datetime'] = df['lpep_dropoff_datetime'].fillna('1970-01-01 00:00:00').astype(str)
df['PULocationID'] = df['PULocationID'].fillna(0).astype(int)
df['DOLocationID'] = df['DOLocationID'].fillna(0).astype(int)
df['passenger_count'] = df['passenger_count'].fillna(1).astype(int)  # Assuming default passenger count as 1
df['trip_distance'] = df['trip_distance'].fillna(0.0).astype(float)
df['tip_amount'] = df['tip_amount'].fillna(0.0).astype(float)

# Start timing
t0 = time()

# Send each row as a message
for _, row in df.iterrows():
    message = row.to_dict()
    producer.send(topic_name, value=message)

# Flush the producer
producer.flush()

# End timing
t1 = time()
print(f"Data sending took {t1 - t0:.2f} seconds")


/var/folders/vd/dl48s8js5_z0k6_rb5jv19xw0000gn/T/ipykernel_25262/1841108983.py:17: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


Data sending took 21.56 seconds


In [18]:
from kafka import KafkaConsumer
import json

# Define topic name
topic_name = "green-trips"

# Initialize Kafka consumer
consumer = KafkaConsumer(
    topic_name,
    bootstrap_servers="localhost:9092",  # Redpanda broker address
    auto_offset_reset="earliest",  # Start from the earliest message
    enable_auto_commit=False,  # Don't commit offsets
    value_deserializer=lambda x: json.loads(x.decode("utf-8")),  # Deserialize JSON messages
)

# Counter to count the messages
message_count = 0

loop = True

while loop:
    messages = consumer.poll(timeout_ms=5000)

    loop = False
    if messages:
        for _, records in messages.items():
            for record in records:
                loop = True
                message_count += 1

# Close the consumer
consumer.close()

# Print the total number of messages consumed
print(f"Total message count: {message_count}")


Total message count: 476386
